In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver import ChromeOptions


In [2]:
str1='https://he.wikipedia.org/wiki/%D7%A2%D7%9E%D7%95%D7%93_%D7%A8%D7%90%D7%A9%D7%99'

In [3]:
str2='https://www.ynet.co.il/home/0,7340,L-8,00.html'

In [4]:
str3='http://www.talniri.co.il/'


In [5]:
urlList=[str1,str2,str3]
urlList

['https://he.wikipedia.org/wiki/%D7%A2%D7%9E%D7%95%D7%93_%D7%A8%D7%90%D7%A9%D7%99',
 'https://www.ynet.co.il/home/0,7340,L-8,00.html',
 'http://www.talniri.co.il/']

In [6]:
#Clean all Tags (Leaves us with only contant)
def CleanTags(soup):
    for script in soup(["script", "style"]):
        script.decompose()

    strips = list(soup.stripped_strings)
    
    words=[]
    for line in strips:
        temp=line.split()
        for word in temp:
            words.append(word)
    
    return words

In [7]:
#Go to website and extract decoded source code
def fetchWebSite(url):
    driver = webdriver.Chrome()
    driver.get(url)
    html_source = (driver.page_source.encode('utf-8'))
    driver.implicitly_wait(30)

    soup = BeautifulSoup(html_source, 'html.parser')
    
    driver.close()
    
    return soup

In [8]:
#genereate list of lists where each list containes all the words from an article
cleanContent=[]
for url in urlList:
    soup=fetchWebSite(url)
    cleanContent.append(CleanTags(soup))
    


In [9]:
#count the number of times each words apeares in all the articles
word_dict={}
for contant in cleanContent:
    my_dict = {i:contant.count(i) for i in contant}
    word_dict.update(my_dict)
    my_dict={}


In [10]:
#Generate DataFrame from the data and sort it by number of repeates
df =pd.DataFrame(list(word_dict.items()),columns = ['word','repeat'])
df=df.sort_values('repeat',ascending=False)



In [11]:
#Clean DataFrame from non word strings
df=df[df['word'].str.isalpha()]

#Add to each word the length of it
df['length'] = df.word.str.len()

#Leave only the words which appeared the most in each legth category
df = df[df.length > 1]
grouped_df = df.groupby("length")
maximums = grouped_df.max()
maximums = maximums.reset_index()
maximums

,length,word,repeat
0,2,贛語,27
1,3,한국어,12
2,4,አማርኛ,17
3,5,ܐܪܡܝܐ,52
4,6,𐌲𐌿𐍄𐌹𐍃𐌺,55
5,7,ᱥᱟᱱᱛᱟᱲᱤ,10
6,8,ئۇيغۇرچە,10
7,9,მარგალური,4
8,10,ⰔⰎⰑⰂⰡⰐⰠⰔⰍⰟ,3
9,11,ואלקטרוניקה,2


In [12]:
#Format it and put it in dictionary as asked
final={}

for i in maximums.length:
    final['length '+str(i)]=maximums.word.iloc[i-2]

final

{'length 2': '贛語',
 'length 3': '한국어',
 'length 4': 'አማርኛ',
 'length 5': 'ܐܪܡܝܐ',
 'length 6': '𐌲𐌿𐍄𐌹𐍃𐌺',
 'length 7': 'ᱥᱟᱱᱛᱟᱲᱤ',
 'length 8': 'ئۇيغۇرچە',
 'length 9': 'მარგალური',
 'length 10': 'ⰔⰎⰑⰂⰡⰐⰠⰔⰍⰟ',
 'length 11': 'ואלקטרוניקה',
 'length 12': 'האנציקלופדיה',
 'length 13': 'האינפלציוניים',
 'length 14': 'בהתחייבויותיהם',
 'length 15': 'Tsetsêhestâhese'}